## GAN

Source: https://github.com/wiseodd/generative-models/tree/master/GAN/vanilla_gan

### setup

In [1]:
import os
import time

In [2]:
import numpy as np

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchnet.meter import AverageValueMeter

In [4]:
import mnist  # for downloading data

In [5]:
epochs = 100
mb_size = 64
lr = 1e-3
X_dim = 784
y_dim = 10
Z_dim = 100
h_dim = 128

intermediate_path = '../intermediate/gan/'
if not os.path.isdir(intermediate_path):
    os.makedirs(intermediate_path)
    
data_path = '../data/mnist/'

torch.manual_seed(7)

### model

In [6]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [7]:
Wzh = xavier_init(size=[Z_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)
Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)
G_params = [Wzh, bzh, Whx, bhx]

def G(z):
    h = F.relu(z @ Wzh + bzh.repeat(z.size(0), 1))
    X = F.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X

In [8]:
Wxh = xavier_init(size=[X_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)
Why = xavier_init(size=[h_dim, 1])
bhy = Variable(torch.zeros(1), requires_grad=True)
D_params = [Wxh, bxh, Why, bhy]

def D(X):
    h = F.relu(X @ Wxh + bxh.repeat(X.size(0), 1))
    y = F.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    return y

### utils

In [9]:
def reset_grad(params):
    for p in params:
        p.grad.data.zero_()

### train

In [10]:
def train(data_loader, G_solver, D_solver, epoch):
    D_losses = AverageValueMeter()
    G_losses = AverageValueMeter()
    start = time.time()
    
    for i, (X, _) in enumerate(data_loader):
        batch_size = X.size(0)
        ones_label = Variable(torch.ones(batch_size))
        zeros_label = Variable(torch.zeros(batch_size))
        X = X.view(-1, X_dim)
        X = Variable(X)
        # Sample data
        z = Variable(torch.randn(X.size(0), Z_dim))
        
        # Dicriminator forward-loss-backward-update
        G_sample = G(z)
        D_real = D(X)
        D_fake = D(G_sample)
        
        D_loss_real = F.binary_cross_entropy(D_real, ones_label)
        D_loss_fake = F.binary_cross_entropy(D_fake, zeros_label)
        
        D_loss = D_loss_real + D_loss_fake
        
        D_loss.backward()
        D_solver.step()
        reset_grad(D_params + G_params)
        
        # Generator forward-loss-backward-update
        z = Variable(torch.randn(X.size(0), Z_dim))
        G_sample = G(z)
        D_fake = D(G_sample)
        
        G_loss = F.binary_cross_entropy(D_fake, ones_label)
        
        G_loss.backward()
        G_solver.step()
        reset_grad(D_params + G_params)
        
        D_losses.add(D_loss.data.cpu()[0] * batch_size, batch_size)
        G_losses.add(G_loss.data.cpu()[0] * batch_size, batch_size)
        
    print("=> EPOCH {} | Time: {}s | D_loss: {:.4f} | G_loss: {:.4f}"
          .format(epoch, round(time.time()-start), 
                  D_losses.value()[0], G_losses.value()[0]))

In [11]:
def generate(epoch):
    z = torch.randn(16, Z_dim)
    samples = G(Variable(z, volatile=True)).data.view(16, 1, 28, 28)
    samples_filename = os.path.join(intermediate_path, 
                                    "{}.png".format(str(epoch).zfill(3)))
    save_image(samples, samples_filename, nrow=4)

### prepare

In [12]:
data_loader = DataLoader(
    mnist.MNIST(data_path, train=True, download=True,
                transform=transforms.ToTensor()),
    batch_size=mb_size, shuffle=True)

In [13]:
G_solver = optim.Adam(G_params, lr=lr)
D_solver = optim.Adam(D_params, lr=lr)

### run

In [14]:
for epoch in range(1, epochs+1):
    train(data_loader, G_solver, D_solver, epoch)
    generate(epoch)

=> EPOCH 1 | Time: 59s | D_loss: 0.0680 | G_loss: 6.2997
=> EPOCH 2 | Time: 53s | D_loss: 0.0147 | G_loss: 6.8657
=> EPOCH 3 | Time: 57s | D_loss: 0.0620 | G_loss: 5.1041
=> EPOCH 4 | Time: 50s | D_loss: 0.1331 | G_loss: 5.1648
=> EPOCH 5 | Time: 51s | D_loss: 0.2408 | G_loss: 4.4107
=> EPOCH 6 | Time: 54s | D_loss: 0.3555 | G_loss: 3.7777
=> EPOCH 7 | Time: 52s | D_loss: 0.4579 | G_loss: 3.4397
=> EPOCH 8 | Time: 57s | D_loss: 0.5595 | G_loss: 3.1252
=> EPOCH 9 | Time: 56s | D_loss: 0.6330 | G_loss: 2.8519
=> EPOCH 10 | Time: 65s | D_loss: 0.6677 | G_loss: 2.5706
=> EPOCH 11 | Time: 68s | D_loss: 0.6616 | G_loss: 2.5322
=> EPOCH 12 | Time: 64s | D_loss: 0.7243 | G_loss: 2.4047
=> EPOCH 13 | Time: 59s | D_loss: 0.7070 | G_loss: 2.5515
=> EPOCH 14 | Time: 61s | D_loss: 0.7544 | G_loss: 2.3858
=> EPOCH 15 | Time: 63s | D_loss: 0.7700 | G_loss: 2.2271
=> EPOCH 16 | Time: 65s | D_loss: 0.7856 | G_loss: 2.1630
=> EPOCH 17 | Time: 64s | D_loss: 0.7999 | G_loss: 2.1053
=> EPOCH 18 | Time: 64s

## DCGAN

Source: https://github.com/pytorch/examples/tree/master/dcgan

### setup

In [1]:
import argparse
import os
import time

In [2]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torchnet.meter import AverageValueMeter

In [3]:
parser = {
    'data_path': '../data/cifar10',
    'epochs': 25,
    'batch_size': 64,
    'lr': 0.0002,
    'image_size': 64,
    'z_dim': 100,
    'G_features': 64,
    'D_features': 64,
    'image_channels': 3,
    'beta1': 0.5,
    'cuda': True,
    'seed': 7,
    'workers': 4,
    'intermediate_path': '../intermediate/dcgan'
}
args = argparse.Namespace(**parser)

args.cuda = args.cuda and torch.cuda.is_available()

if not os.path.isdir(args.data_path):
    os.makedirs(args.data_path)
if not os.path.isdir(args.intermediate_path):
    os.makedirs(args.intermediate_path)
    
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

### model

In [4]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(args.z_dim, args.G_features * 8,
                               4, 1, 0, bias=False),
            nn.BatchNorm2d(args.G_features * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(args.G_features * 8, args.G_features * 4,
                               4, 2, 1, bias=False),
            nn.BatchNorm2d(args.G_features * 4),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(args.G_features * 4, args.G_features * 2,
                               4, 2, 1, bias=False),
            nn.BatchNorm2d(args.G_features * 2),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(args.G_features * 2, args.G_features,
                               4, 2, 1, bias=False),
            nn.BatchNorm2d(args.G_features),
            nn.ReLU(True),
            # state size. (ngf) x 16 x 16
            nn.ConvTranspose2d(args.G_features, args.image_channels,
                               4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 32 x 32
        )
        self._initialize_weights()
        
    # custom weight initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.ConvTranspose2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                init.normal(m.weight, mean=0, std=0.02)
            elif isinstance(m, nn.BatchNorm2d):
                init.normal(m.weight, mean=1, std=0.02)
                init.constant(m.bias, 0)

    def forward(self, x):
        return self.main(x)

In [5]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 32 x 32
            nn.Conv2d(args.image_channels, args.D_features,
                      4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 16 x 16
            nn.Conv2d(args.D_features, args.D_features * 2,
                      4, 2, 1, bias=False),
            nn.BatchNorm2d(args.D_features * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 8 x 8
            nn.Conv2d(args.D_features * 2, args.D_features * 4,
                      4, 2, 1, bias=False),
            nn.BatchNorm2d(args.D_features * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(args.D_features * 4, args.D_features * 8,
                      4, 2, 1, bias=False),
            nn.BatchNorm2d(args.D_features * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(args.D_features * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        self._initialize_weights()
        
    # custom weight initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                init.normal(m.weight, mean=0, std=0.02)
            elif isinstance(m, nn.BatchNorm2d):
                init.normal(m.weight, mean=1, std=0.02)
                init.constant(m.bias, 0)

    def forward(self, input):
        output = self.main(input)
        return output.view(-1, 1)

### train

In [6]:
def train(args, data_loader, netG, netD, G_optimizer, D_optimizer,
          criterion, epoch):
    D_losses = AverageValueMeter()
    G_losses = AverageValueMeter()
    D_reals = AverageValueMeter()
    D_fakes = AverageValueMeter()
    G_reals = AverageValueMeter()
    
    start = time.time()
    # call Variable after .cuda()
    for i, (real, _) in enumerate(data_loader):
        batch_size = real.size(0)
        real_label = torch.ones(batch_size)
        fake_label = torch.zeros(batch_size)
        z = torch.randn(batch_size, args.z_dim, 1, 1)
        if args.cuda:
            real_label = real_label.cuda()
            fake_label = fake_label.cuda()
            real = real.cuda()
            z = z.cuda()
        real_label = Variable(real_label)
        fake_label = Variable(fake_label)
        real = Variable(real)
        z = Variable(z)
        
        real_output = netD(real)
        D_real_loss = criterion(real_output, real_label)
        D_real = real_output.data.mean()
        
        fake = netG(z)
        fake_output = netD(fake.detach())
        D_fake_loss = criterion(fake_output, fake_label)
        D_fake = fake_output.data.mean()
        
        D_loss = D_real_loss + D_fake_loss
        netD.zero_grad()
        D_loss.backward()
        D_optimizer.step()
        
        output = netD(fake)
        G_loss = criterion(output, real_label)
        G_real = output.data.mean()
        netG.zero_grad()
        G_loss.backward()
        G_optimizer.step()
        
        D_losses.add(D_loss.data.cpu()[0] * batch_size, batch_size)
        G_losses.add(G_loss.data.cpu()[0] * batch_size, batch_size)
        D_reals.add(D_real * batch_size, batch_size)
        D_fakes.add(D_fake * batch_size, batch_size)
        G_reals.add(G_real * batch_size, batch_size)
        
    print("=> EPOCH {} | Time: {}s | D_loss: {:.4f} | G_loss: {:.4f}"
          " | D_real: {:.4f} | D_fake: {:.4f} | G_real: {:.4f}"
          .format(epoch, round(time.time()-start), D_losses.value()[0],
                  G_losses.value()[0], D_reals.value()[0],
                  D_fakes.value()[0], G_reals.value()[0]))

In [7]:
def generate(args, netG, epoch):
    z = torch.randn(args.batch_size, args.z_dim, 1, 1)
    if args.cuda:
        z = z.cuda()
    fake = netG(Variable(z, volatile=True))
    save_image(fake.data.cpu(), os.path.join(args.intermediate_path,
        "fake_sample_epoch_{:02d}.png".format(epoch)), normalize=True)

### prepare

In [8]:
dataset = datasets.CIFAR10(root=args.data_path, download=True,
                           transform=transforms.Compose([
                               transforms.Scale(args.image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5))]))
data_loader = DataLoader(dataset, batch_size=args.batch_size,
                         shuffle=True,
                         num_workers=args.workers)

Files already downloaded and verified


In [9]:
netG = _netG()
netD = _netD()
criterion = nn.BCELoss()
if args.cuda:
    netD.cuda()
    netG.cuda()
    criterion.cuda()

In [10]:
D_optimizer = optim.Adam(netD.parameters(), lr=args.lr,
                         betas=(args.beta1, 0.999))
G_optimizer = optim.Adam(netG.parameters(), lr=args.lr,
                         betas=(args.beta1, 0.999))

In [11]:
sample, _ = iter(data_loader).next()
save_image(sample, os.path.join(args.intermediate_path, "real_sample.png"))

### run

In [12]:
for epoch in range(1, args.epochs+1):
    train(args, data_loader, netG, netD, G_optimizer, D_optimizer,
          criterion, epoch)
    generate(args, netG, epoch)

=> EPOCH 1 | Time: 73s | D_loss: 0.5287 | G_loss: 7.2922 | D_real: 0.8419 | D_fake1: 0.1556 | D_fake2: 0.0327
=> EPOCH 2 | Time: 73s | D_loss: 0.6256 | G_loss: 4.0059 | D_real: 0.7921 | D_fake1: 0.2034 | D_fake2: 0.0586
=> EPOCH 3 | Time: 73s | D_loss: 0.6680 | G_loss: 3.3827 | D_real: 0.7782 | D_fake1: 0.2193 | D_fake2: 0.0919
=> EPOCH 4 | Time: 73s | D_loss: 0.7120 | G_loss: 3.2519 | D_real: 0.7685 | D_fake1: 0.2298 | D_fake2: 0.1092
=> EPOCH 5 | Time: 73s | D_loss: 0.6155 | G_loss: 3.5447 | D_real: 0.8067 | D_fake1: 0.1925 | D_fake2: 0.0912
=> EPOCH 6 | Time: 73s | D_loss: 0.5895 | G_loss: 3.7285 | D_real: 0.8215 | D_fake1: 0.1778 | D_fake2: 0.0806
=> EPOCH 7 | Time: 73s | D_loss: 0.6860 | G_loss: 3.3466 | D_real: 0.7905 | D_fake1: 0.2082 | D_fake2: 0.1085
=> EPOCH 8 | Time: 73s | D_loss: 0.5958 | G_loss: 3.5248 | D_real: 0.8168 | D_fake1: 0.1809 | D_fake2: 0.0961
=> EPOCH 9 | Time: 73s | D_loss: 0.6963 | G_loss: 3.2522 | D_real: 0.7886 | D_fake1: 0.2098 | D_fake2: 0.1126
=> EPOCH 1

In [13]:
torch.save(netG.state_dict(), os.path.join(args.intermediate_path,
                                           "netG.pth"))